In [4]:
import os
import requests
import pandas as pd

In [5]:
trawlers = pd.read_csv('../raw_data/trawlers.csv')
trawlers.shape

(4369101, 10)

In [6]:
trawlers.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200,-1.0,gfw
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167,-1.0,gfw
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183,-1.0,gfw


In [7]:
trawlers['type']='trawlers'

In [8]:
longlines = pd.read_csv('../raw_data/drifting_longlines.csv')
longlines.shape

(13968727, 10)

In [9]:
longlines.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.263956e+13,1.327137e+09,232994.281250,311748.65625,8.2,230.500000,14.865583,-26.853662,-1.0,dalhousie_longliner
1,1.263956e+13,1.327137e+09,233994.265625,312410.34375,7.3,238.399994,14.863870,-26.856800,-1.0,dalhousie_longliner
2,1.263956e+13,1.327137e+09,233994.265625,312410.34375,6.8,238.899994,14.861551,-26.860649,-1.0,dalhousie_longliner


In [10]:
longlines['mmsi'].nunique()

110

In [11]:
longlines['type']='longlines'

In [12]:
gear = pd.read_csv('../raw_data/fixed_gear.csv')
gear.shape

(1559137, 10)

In [13]:
gear.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,7.572519e+12,1.347664e+09,0.0,36054.625000,0.0,0.000000,42.798748,-8.944992,-1.0,gfw
1,7.572519e+12,1.348056e+09,0.0,36054.625000,0.0,0.000000,42.798717,-8.945075,-1.0,gfw
2,7.572519e+12,1.350409e+09,0.0,90970.296875,0.0,198.199997,43.106419,-9.215466,-1.0,gfw


In [14]:
gear['mmsi'].nunique()

36

In [15]:
gear['type']='gear'

In [16]:
pole = pd.read_csv('../raw_data/pole_and_line.csv')
pole.shape

(161315, 10)

In [17]:
pole.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.848346e+13,1.340882e+09,0.0,2236.013184,0.0,0.000000,28.967354,-13.537797,-1.0,gfw
1,1.848346e+13,1.340884e+09,0.0,2236.013184,0.0,125.199997,28.967373,-13.537838,-1.0,gfw
2,1.848346e+13,1.340885e+09,0.0,2236.013184,0.0,0.000000,28.967354,-13.537838,-1.0,gfw


In [18]:
pole['mmsi'].nunique()

6

In [19]:
pole['type']='pole'

In [20]:
seines = pd.read_csv('../raw_data/purse_seines.csv')
seines.shape

(1545323, 10)

In [21]:
seines.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,9.924005e+12,1.379601e+09,0.0,1414.178833,0.0,298.500000,8.861500,-79.668427,-1.0,false_positives
1,9.924005e+12,1.379602e+09,0.0,1414.178833,0.0,298.500000,8.861506,-79.668442,-1.0,false_positives
2,9.924005e+12,1.379604e+09,0.0,1414.178833,0.1,128.399994,8.861511,-79.668488,-1.0,false_positives


In [22]:
seines['mmsi'].nunique()

28

In [23]:
seines['type']='seines'

In [24]:
trollers = pd.read_csv('../raw_data/trollers.csv')
trollers.shape

(166243, 10)

In [25]:
trollers.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,7.652701e+13,1.337836e+09,0.0,3162.200195,0.0,0.000000,51.887592,4.356583,-1.0,gfw
1,7.652701e+13,1.338199e+09,0.0,4999.877441,0.0,0.000000,51.242130,4.403008,-1.0,gfw
2,7.652701e+13,1.343752e+09,0.0,66308.250000,8.6,292.200012,51.960873,5.196125,-1.0,gfw


In [26]:
trollers['mmsi'].nunique()

5

In [27]:
trollers['type']='trollers'

In [28]:
unknown = pd.read_csv('../raw_data/unknown.csv')
unknown.shape

(6811552, 10)

In [29]:
unknown.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.833071e+14,1.343786e+09,314242.1875,538727.93750,13.1,62.700001,2.230797,157.382812,-1.0,crowd_sourced
1,1.833071e+14,1.343786e+09,314242.1875,538727.93750,13.8,65.199997,2.232352,157.386047,-1.0,crowd_sourced
2,1.833071e+14,1.343792e+09,343947.9375,513526.09375,13.0,61.700001,2.410787,157.745605,-1.0,crowd_sourced


In [30]:
unknown['mmsi'].nunique()

120

In [31]:
unknown['type']='unknown'

In [38]:
list_dfs = [trawlers, longlines, gear, pole, seines, trollers, unknown]
df = pd.concat(list_dfs)
df.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source,type
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200,-1.0,gfw,trawlers
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167,-1.0,gfw,trawlers
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183,-1.0,gfw,trawlers


In [39]:
df['mmsi'].nunique()

354

In [40]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse_output =False)

# Fit encoder
ohe.fit(df[['type']])

# Transform the current "Alley" column
df[ohe.get_feature_names_out()] = ohe.transform(df[['type']])

# Drop the column "Alley" which has been encoded
df.drop(columns = ["type"], inplace = True)

In [41]:
df.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source,type_gear,type_longlines,type_pole,type_seines,type_trawlers,type_trollers,type_unknown
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200,-1.0,gfw,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167,-1.0,gfw,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183,-1.0,gfw,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [42]:
df.drop(columns=['source'], inplace=True)
df.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,type_gear,type_longlines,type_pole,type_seines,type_trawlers,type_trollers,type_unknown
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [44]:
df_fishing = df.loc[df['is_fishing'] > -1]

In [45]:
df_fishing['is_fishing'].value_counts()

is_fishing
0.000000    295979
1.000000    247498
0.666667      4806
0.333333      4096
0.750000       752
0.250000       670
0.800000        33
0.166667        12
0.400000         9
Name: count, dtype: int64

In [46]:
df_fishing['is_fishing'] = round(df_fishing['is_fishing'])

/var/folders/jp/3hlxbh4x1mb0k2qwf5fmzyp80000gp/T/ipykernel_78188/3636107561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fishing['is_fishing'] = round(df_fishing['is_fishing'])


In [47]:
df_fishing['is_fishing'].value_counts()

is_fishing
0.0    300766
1.0    253089
Name: count, dtype: int64